In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)

# Anonymisation

Here I anonymise the synethic data from our model.


## TOC
 * [Preliminary](#Preliminary)
 * [Transaction Details](#Transaction-Details)
   * [Amount](#Amount)
   * [Time](#Time)
 * [Customer Details](#Customer-Details)
   * [Credit Card Number](#Credit-Card-Number)
   * [Person Details](#Person-Details)
   * [Other](#Other)
 * [Merchant Details](#Merchant-Details)
 * [Location Details](#Location-Details)

In [2]:
# Load the data
training_data=pd.read_csv("data/synthetic_train.csv",index_col=0)
test_data=pd.read_csv("data/synthetic_train.csv",index_col=0)
full_data=pd.concat([training_data, test_data])

sample_df=full_data.sample(n=10)

c:\users\domah\appdata\local\programs\python\python37\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
print(full_data.shape)
full_data.dtypes

(2593350, 22)


trans_date_trans_time     object
cc_num                     int64
merchant                  object
category                  object
amt                      float64
first                     object
last                      object
gender                    object
street                    object
city                      object
state                     object
zip                        int64
lat                      float64
long                     float64
city_pop                   int64
job                       object
dob                       object
trans_num                 object
unix_time                  int64
merch_lat                float64
merch_long               float64
is_fraud                   int64
dtype: object

In [4]:
# Sample of the data
sample_df.head(n=10)

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
683415,2019-10-19 00:31:17,4561892980175,"fraud_Hermiston, Russel and Price",food_dining,6.46,Jacqueline,Allen,F,166 Lisa Fords,Greenwood,IN,46143,39.5960,-86.1309,78968,Financial adviser,1968-02-10,f6b15277cfb43198703aa9c94ff7da76,1350606677,40.581341,-87.108239,0
5710,2019-01-04 17:12:25,30518206766474,"fraud_Hahn, Bahringer and McLaughlin",personal_care,8.00,Tamara,Martinez,F,471 Marquez Prairie Suite 680,Matthews,MO,63867,36.7154,-89.6287,1019,Aeronautical engineer,1979-01-26,e2e5715174b4978b6dc3d78aa9c80b27,1325697145,36.980240,-89.069913,0
716370,2019-11-02 12:57:40,2296006538441789,fraud_Zulauf LLC,personal_care,9.50,Judy,Hogan,F,4970 Michelle Burgs,Brooklyn,NY,11217,40.6816,-73.9798,2504700,Medical sales representative,1999-09-01,0ef24ccc2a5a24348f1036699560cfd9,1351861060,41.359175,-74.549937,0
681614,2019-10-18 01:54:34,213112402583773,fraud_Koepp-Witting,grocery_pos,80.03,Ana,Howell,F,4664 Sanchez Common Suite 930,Bradley,SC,29819,34.0326,-82.2027,1523,Research scientist (physical sciences),1984-06-03,4a3ddfa9f69e5c8081f2c21d8a7e8d6e,1350525274,33.036601,-83.139192,0
395705,2019-06-29 22:21:01,6539653032686894,fraud_Dietrich-Fadel,health_fitness,189.57,Samuel,Short,M,23383 Denise Pine Apt. 099,Sacramento,CA,95842,38.6865,-121.3494,757530,Soil scientist,1966-09-19,da374b22c8ecdc5eebe1f8e12f9545f2,1341008461,37.947820,-122.294495,0
1033859,2020-03-04 02:14:28,345389171551808,fraud_Emmerich-Luettgen,grocery_net,50.33,Justin,Fowler,M,5569 Phillips Neck Apt. 003,Coffeeville,MS,38922,33.9215,-89.6782,3451,Financial trader,1984-05-19,39831f85af13f73f9a600a4d3d7d1108,1362363268,33.621432,-89.457708,0
872613,2019-12-20 22:26:35,374125201044065,fraud_Fadel Inc,health_fitness,87.55,Christopher,Gilbert,M,20937 Reed Lakes Apt. 271,Washington,DC,20012,38.9757,-77.0282,601723,"Optician, dispensing",1970-07-20,6fbde8ab60f52dc70565518d1c7ec42f,1356042395,38.675350,-76.719537,0
919168,2019-12-30 14:43:00,4671727014157745,"fraud_O'Reilly, Mohr and Purdy",home,44.15,Kenneth,Edwards,M,3653 Ryan Crossroad,Andrews,IN,46702,40.8618,-85.6067,2304,Retail banker,1955-07-25,5ebce043f0a8a165160e7974313210e2,1356878580,41.252948,-85.262964,0
523329,2019-08-13 13:44:41,4464457352619,"fraud_Yost, Schamberger and Windler",kids_pets,8.31,Breanna,Rodriguez,F,118 Cabrera Springs Apt. 105,Lanark Village,FL,32323,29.8826,-84.5964,217,Television production assistant,1990-01-24,d9337f9b69265c1ab36bb48fbdd25073,1344865481,29.404878,-83.942956,0
345465,2019-06-11 12:05:45,4334230547694630,"fraud_Durgan, Gislason and Spencer",home,288.37,Scott,Martin,M,7483 Navarro Flats,Freedom,WY,83120,43.0172,-111.0292,471,"Education officer, museum",1967-08-02,c33d84c9e2eb8f6d7eac6b901111c8eb,1339416345,43.290325,-110.841566,0


## Preliminary

Convert `trans_date_trans_time` and `dob` into `datetime` objects.

In [5]:
full_data["trans_date_trans_time"]=pd.to_datetime(full_data["trans_date_trans_time"],format="%Y-%m-%d %H:%M:%S")
full_data["dob"]=pd.to_datetime(full_data["dob"],format="%Y-%m-%d")

sample_df["trans_date_trans_time"]=pd.to_datetime(sample_df["trans_date_trans_time"],format="%Y-%m-%d %H:%M:%S")
sample_df["dob"]=pd.to_datetime(sample_df["dob"],format="%Y-%m-%d")

**Cleaned Data**

I define the dataframe `clean_df` which will hold all the prepared data

In [6]:
clean_df=pd.DataFrame()
clean_df["is_fraud"]=sample_df["is_fraud"]

## Transcation Details
 * `trans_date_trans_time`
 * `amt`
 * `trans_num`
 * `unix_time`
 * `is_fraud`
 
Leave `is_fraud` untouched as it is the feature we will be fitting the model to.

## Amount

In [8]:
clean_df["amt"]=sample_df["amt"].copy()

## Time

In the real data set time is given in seconds since the first transaction. I implement the function `standardise_time` which calculates the number of seconds between mid-night on the day of the first transaction and each transaction.

The full-time columns (`trans_date_trans_time` and `unix_time`) are still useful as you can use them to determine the year, month, day etc. of the transaction and thus may want to consider how to incorporate these into feature selection. 

In [9]:
def standardise_time(series) -> pd.Series:
    min_time=series.min().to_pydatetime()
    min_day=min_time.replace(second=0,minute=0,hour=0)
    return ((series-min_day).dt.total_seconds()).astype(int)

In [10]:
clean_df["unix_time"]=sample_df["unix_time"].copy()

## Customer Details

**Account Details**
 * `cc_num`
 
**Person Details**
 * `first`
 * `last`
 * `gender`
 * Address (`street`, `city`, `state`, `zip`, `lat`, `long`)
  * *Owen is working on this as we want to maintain relationships between locations*. (See [below](#Location-Details))
 * `job`
 * `dob`
 
**Other**
 * `city_pop`

### Credit Card Number

Anonymise credit card number

In [11]:
def anonymise(series:pd.Series) -> pd.Series:
    return series.astype("category").cat.codes

In [12]:
clean_df["cc_id"]=anonymise(sample_df["cc_num"])
clean_df.head(1)

,is_fraud,amt,unix_time,cc_id
683415,0,6.46,1350606677,1


### Personal Details

`first` and `last` contains the customers name, these need to be anonymised. I introduce the column `person_id` which contains a unique identifier for each person in the dataset (It is assumed each unique combination of `first`, `last`, `job` and `dob` is a unique person).

In [13]:
clean_df["person_id"]=anonymise(sample_df["first"]+"_"+sample_df["last"]+"_"+sample_df["job"]+"_"+sample_df["dob"].apply(lambda x: x.strftime('%Y-%m-%d')))

Keeping `gender` seems reasonable as it may be useful in the model fit. May be worth further investigation into whether there are notable differences in fraud rates/patterns between the genders.

In [14]:
clean_df["gender_id"]=anonymise(sample_df["gender"])

`job` could be useful for the model and does not necessarily need anonymising. I will leave it as is and likely try to group jobs during feature selection to reduce degrees of freedom.

In [15]:
clean_df["job_category"]=sample_df["job"]

`dob` should be useful for the model, I am going to discretise this data into the age of the customer at the time of the transaction.

In [16]:
def dob_to_age(df) -> pd.Series:
    return (df["trans_date_trans_time"]-df["dob"])//np.timedelta64(1,"Y")

In [17]:
clean_df["age"]=dob_to_age(sample_df[["dob","trans_date_trans_time"]])

Intuitively it seems likely that `city_pop` may be useful in fitting the model, so we want to keep it. However, it could be used to identify the city the customer is from, due to the precise nature of this value, thus I round it up to the nearest 50.

During feature selection, it is likely we would want to discretise/categorise `city_pop` values. This will introduce our prior knowledge and reduce the degrees of freedom which the model needs to consider.

In [18]:
clean_df["city_pop_round"]=50*np.floor(sample_df["city_pop"]/50).copy().astype(int)

In [19]:
clean_df.head(5)

,is_fraud,amt,unix_time,cc_id,person_id,gender_id,job_category,age,city_pop_round
683415,0,6.46,1350606677,1,3,0,Financial adviser,51,78950
5710,0,8.00,1325697145,2,9,0,Aeronautical engineer,39,1000
716370,0,9.50,1351861060,6,4,0,Medical sales representative,20,2504700
681614,0,80.03,1350525274,3,0,0,Research scientist (physical sciences),35,1500
395705,0,189.57,1341008461,9,7,1,Soil scientist,52,757500


## Merchant Details
 * `merchant`
 * `category`
 * Location (`merch_lat`,`merch_long`)
   * *Owen is working on this as we want to maintain relationships between locations*. (See [below](#Location-Details))

`merchant` is the name of the merchant and should be anonymised. This is done by simple converting it into an id.

In [20]:
clean_df["merchant_id"]=anonymise(sample_df["merchant"])

`category` is the verbose category of the merchant. This does not necessarily need to be anonymised as it is not identifiable, but some pre-processing should be performed in order to reduce the degrees of freedom of the model.

In [21]:
clean_df["merchant_category"]=sample_df["category"]

In [22]:
clean_df.head(5)

,is_fraud,amt,unix_time,cc_id,person_id,gender_id,job_category,age,city_pop_round,merchant_id,merchant_category
683415,0,6.46,1350606677,1,3,0,Financial adviser,51,78950,5,food_dining
5710,0,8.00,1325697145,2,9,0,Aeronautical engineer,39,1000,4,personal_care
716370,0,9.50,1351861060,6,4,0,Medical sales representative,20,2504700,9,personal_care
681614,0,80.03,1350525274,3,0,0,Research scientist (physical sciences),35,1500,6,grocery_pos
395705,0,189.57,1341008461,9,7,1,Soil scientist,52,757500,0,health_fitness


## Location Details

There is location data for customers (`street`, `city`, `state`, `zip`, `lat`, `long`) and for merchants (`merch_lat`,`merch_long`).

*Owen is looking at this*

## Save File

In [23]:
def save_data(df:pd.DataFrame,file_path):
    df.to_csv(file_path)

In [24]:
# save_data("","data/prepared_syntheticg_data.csv")